## Crawl Tweets

In [9]:
!pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
from tweepy import OAuthHandler, API, Cursor

In [56]:
with open('twitter_token3.json','r') as f:
  token = json.load(f)

In [11]:
token.keys()

dict_keys(['consumer_key', 'consumer_secret', 'access_token', 'access_secret'])

In [57]:
access_token = token['access_token']
access_secret = token['access_secret']
consumer_key = token['consumer_key']
consumer_secret = token['consumer_secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = API(auth, wait_on_rate_limit = True, timeout = 15*60)

In [58]:
## Jika terdapat lebbih dari 100 tweet dan kita ingin dapatkan semua tweet tersebut, bisa pakai fungsi/method Cursor dari tweepy

tweets_all = []

## keyword yg akan kita cari dari sebuah tweet
query = '"mobil listrik"'

count = 1
for tweet_list in Cursor(api.search, query, count=100, tweet_mode='extended', lang='id').pages():
    tweets_all.extend([i._json for i in tweet_list])
    print('request ke:',count)
    count+=1

request ke: 1
request ke: 2
request ke: 3
request ke: 4
request ke: 5
request ke: 6
request ke: 7
request ke: 8
request ke: 9
request ke: 10
request ke: 11
request ke: 12
request ke: 13
request ke: 14
request ke: 15
request ke: 16
request ke: 17
request ke: 18
request ke: 19
request ke: 20
request ke: 21
request ke: 22
request ke: 23
request ke: 24
request ke: 25
request ke: 26
request ke: 27
request ke: 28
request ke: 29
request ke: 30
request ke: 31
request ke: 32
request ke: 33
request ke: 34
request ke: 35
request ke: 36
request ke: 37
request ke: 38
request ke: 39
request ke: 40
request ke: 41
request ke: 42
request ke: 43
request ke: 44
request ke: 45
request ke: 46
request ke: 47
request ke: 48
request ke: 49
request ke: 50
request ke: 51
request ke: 52
request ke: 53
request ke: 54
request ke: 55
request ke: 56
request ke: 57
request ke: 58
request ke: 59
request ke: 60
request ke: 61
request ke: 62
request ke: 63
request ke: 64
request ke: 65
request ke: 66
request ke: 67
requ

In [59]:
print(len(tweets_all))

9298


In [60]:
tweets_all[0]

{'created_at': 'Fri Jan 27 13:19:35 +0000 2023',
 'id': 1618961923156115457,
 'id_str': '1618961923156115457',
 'full_text': 'Gunakan PLN Mobile Terus, Siapa Tahu Menang Undian Mobil Listrik Minggu Ini! https://t.co/sd5PIowrz8 https://t.co/jOmJ28DuV8',
 'truncated': False,
 'display_text_range': [0, 100],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/sd5PIowrz8',
    'expanded_url': 'http://dlvr.it/ShXCMH',
    'display_url': 'dlvr.it/ShXCMH',
    'indices': [77, 100]}],
  'media': [{'id': 1618961920249458688,
    'id_str': '1618961920249458688',
    'indices': [101, 124],
    'media_url': 'http://pbs.twimg.com/media/Fne1SfFacAAyEHK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/Fne1SfFacAAyEHK.jpg',
    'url': 'https://t.co/jOmJ28DuV8',
    'display_url': 'pic.twitter.com/jOmJ28DuV8',
    'expanded_url': 'https://twitter.com/irwanmapparenta/status/1618961923156115457/photo/1',
    'type': 'photo',
    'sizes': {'thumb

In [61]:
from google.colab import drive

drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [21]:
with open('/content/drive/MyDrive/Intro SNA/Hands-on Data/tweets_#IndonesiaMasters2023.json','w') as f:
    json.dump(tweets_all,f)

### Kalau belum ada twitter API nya, bisa langsung ke generate data SNA nya saja

## Generate data SNA

In [22]:
import pandas as pd
from datetime import datetime, timedelta
from google.colab import drive
import json

In [23]:
# drive.mount('drive')

In [ ]:
## yyyy-mm-dd h:m:s

In [25]:
# tweets_all[1]['retweeted_status']

In [26]:
def get_tweet_details(x, tweet_list):
    
    user = x['user']['screen_name']
    user_id = x['user']['id_str']
    _id = x['id_str']
    date = x['created_at']
    text = x['full_text']
    post_url = 'https://twitter.com/' + user + '/status/' + _id
    
            
    dct = {'source':user, 'user_id':user_id, 'tweet_id':_id, 'created_at':date, 'text':text, 'post_url':post_url, 
           'target':'', 'target_user_id':'', 'target_tweet':''}
    
    if 'retweeted_status' in x.keys():
        target_user = x['retweeted_status']['user']['screen_name']
        target_tweet = x['retweeted_status']['id_str']
        dct['target'] = target_user
        dct['target_user_id'] = x['retweeted_status']['user']['id_str']
        dct['target_tweet'] = 'https://twitter.com/{}/status/{}'.format(target_user, str(target_tweet))
        dct['tweet_type'] = 'Retweet'
        tweet_list.append(dct)
    
    elif 'quoted_status' in x.keys():
        target_user = x['quoted_status']['user']['screen_name']
        target_tweet = x['quoted_status']['id_str']
        dct['target'] = x['quoted_status']['user']['screen_name']
        dct['target_user_id'] = x['quoted_status']['user']['id_str']
        dct['target_tweet'] = 'https://twitter.com/{}/status/{}'.format(target_user, str(target_tweet))
        dct['tweet_type'] = 'Quote'
        tweet_list.append(dct)
        
    if x['in_reply_to_screen_name'] != None:
        target_user = x['in_reply_to_screen_name']
        target_tweet = x['in_reply_to_status_id_str']
        dct['target'] = target_user
        dct['target_user_id'] = x['in_reply_to_user_id_str']
        dct['target_tweet'] = 'https://twitter.com/{}/status/{}'.format(target_user, str(target_tweet))
        dct['tweet_type'] = 'Reply'
        tweet_list.append(dct)
    
    if all(k not in x.keys() for k in ['retweeted_status','quoted_status']) and x['in_reply_to_screen_name'] == None:
        dct['tweet_type'] = 'Tweet'
        tweet_list.append(dct)

In [ ]:
data = []

with open('tweets_pacmannai.json','r') as f:
    data = json.load(f)

In [62]:
data = tweets_all.copy()
len(data)

9298

In [63]:
tweets = []

for elem in data:
    get_tweet_details(elem,tweets)

In [64]:
tweets[0]

{'source': 'irwanmapparenta',
 'user_id': '567950720',
 'tweet_id': '1618961923156115457',
 'created_at': 'Fri Jan 27 13:19:35 +0000 2023',
 'text': 'Gunakan PLN Mobile Terus, Siapa Tahu Menang Undian Mobil Listrik Minggu Ini! https://t.co/sd5PIowrz8 https://t.co/jOmJ28DuV8',
 'post_url': 'https://twitter.com/irwanmapparenta/status/1618961923156115457',
 'target': '',
 'target_user_id': '',
 'target_tweet': '',
 'tweet_type': 'Tweet'}

In [65]:
tweets_df = pd.json_normalize(tweets)
tweets_df.head()

,source,user_id,tweet_id,created_at,text,post_url,target,target_user_id,target_tweet,tweet_type
0,irwanmapparenta,567950720,1618961923156115457,Fri Jan 27 13:19:35 +0000 2023,"Gunakan PLN Mobile Terus, Siapa Tahu Menang Un...",https://twitter.com/irwanmapparenta/status/161...,,,,Tweet
1,auradhamira,42177588,1618959318631743488,Fri Jan 27 13:09:15 +0000 2023,RT @adriansyahyasin: Amsterdam membuka parkir ...,https://twitter.com/auradhamira/status/1618959...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet
2,dimasoemardjono,356926514,1618958812068839425,Fri Jan 27 13:07:14 +0000 2023,RT @adriansyahyasin: Amsterdam membuka parkir ...,https://twitter.com/dimasoemardjono/status/161...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet
3,nama_pendek,840857303364509696,1618958739914235905,Fri Jan 27 13:06:57 +0000 2023,RT @adriansyahyasin: Ini lah\n\nOrang banyak b...,https://twitter.com/nama_pendek/status/1618958...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet
4,makmumnyajinu,3248489016,1618958573211639808,Fri Jan 27 13:06:17 +0000 2023,RT @adriansyahyasin: Amsterdam membuka parkir ...,https://twitter.com/makmumnyajinu/status/16189...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet


In [66]:
month_dct = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04',"May":'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10',
            'Nov':'11','Dec':'12'}

def get_created_at(x):
    
    _ls = x.split()
    date = _ls[2]
    month = _ls[1]
    month = month_dct.get(month)
    year = _ls[-1]
        
    new_date = ('-').join([year,month,date])
    new_date = new_date + ' ' + _ls[3][:8]
    new_date = datetime.strptime(new_date, '%Y-%m-%d %H:%M:%S')+timedelta(hours=7)
    
    return new_date

In [67]:
tweets_df['created_at'] = tweets_df['created_at'].apply(lambda x: get_created_at(x))
tweets_df.head()

,source,user_id,tweet_id,created_at,text,post_url,target,target_user_id,target_tweet,tweet_type
0,irwanmapparenta,567950720,1618961923156115457,2023-01-27 20:19:35,"Gunakan PLN Mobile Terus, Siapa Tahu Menang Un...",https://twitter.com/irwanmapparenta/status/161...,,,,Tweet
1,auradhamira,42177588,1618959318631743488,2023-01-27 20:09:15,RT @adriansyahyasin: Amsterdam membuka parkir ...,https://twitter.com/auradhamira/status/1618959...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet
2,dimasoemardjono,356926514,1618958812068839425,2023-01-27 20:07:14,RT @adriansyahyasin: Amsterdam membuka parkir ...,https://twitter.com/dimasoemardjono/status/161...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet
3,nama_pendek,840857303364509696,1618958739914235905,2023-01-27 20:06:57,RT @adriansyahyasin: Ini lah\n\nOrang banyak b...,https://twitter.com/nama_pendek/status/1618958...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet
4,makmumnyajinu,3248489016,1618958573211639808,2023-01-27 20:06:17,RT @adriansyahyasin: Amsterdam membuka parkir ...,https://twitter.com/makmumnyajinu/status/16189...,adriansyahyasin,169445636,https://twitter.com/adriansyahyasin/status/161...,Retweet


In [71]:
edge = tweets_df.copy()
edge.columns = ['Source Label', 'Source', 'Tweet Id','Created at','Text','Post Url','Target Label','Target','Target Tweet','Tweet Type']
edge['Source'] = edge['Source Label'].copy()
edge['Target'] = edge['Target Label'].copy()
edge = edge.sort_values('Created at').reset_index(drop=True)
# print(len(edge))
edge = edge[(edge['Created at']>=datetime.strptime("2023-01-19",'%Y-%m-%d')) & (edge['Created at']<=datetime.strptime('2023-01-26','%Y-%m-%d'))].reset_index(drop=True)
edge = edge[(edge['Target']!='') & (edge['Tweet Type']!='Reply')].reset_index(drop=True)
print(len(edge))
edge.head()

5624


,Source Label,Source,Tweet Id,Created at,Text,Post Url,Target Label,Target,Target Tweet,Tweet Type
0,wanderfulgerll,wanderfulgerll,1615756013147410432,2023-01-19 00:00:27,RT @adriansyahyasin: Macet ...,https://twitter.com/wanderfulgerll/status/1615...,adriansyahyasin,adriansyahyasin,https://twitter.com/adriansyahyasin/status/161...,Retweet
1,dennyidi,dennyidi,1615756108798521347,2023-01-19 00:00:50,RT @adriansyahyasin: Macet ...,https://twitter.com/dennyidi/status/1615756108...,adriansyahyasin,adriansyahyasin,https://twitter.com/adriansyahyasin/status/161...,Retweet
2,ilhammufc,ilhammufc,1615756123512135683,2023-01-19 00:00:53,RT @adriansyahyasin: Macet ...,https://twitter.com/ilhammufc/status/161575612...,adriansyahyasin,adriansyahyasin,https://twitter.com/adriansyahyasin/status/161...,Retweet
3,icasaurus,icasaurus,1615756145125396481,2023-01-19 00:00:58,RT @adriansyahyasin: Macet ...,https://twitter.com/icasaurus/status/161575614...,adriansyahyasin,adriansyahyasin,https://twitter.com/adriansyahyasin/status/161...,Retweet
4,gajahahahahaha,gajahahahahaha,1615756323177787393,2023-01-19 00:01:41,RT @adriansyahyasin: Macet ...,https://twitter.com/gajahahahahaha/status/1615...,adriansyahyasin,adriansyahyasin,https://twitter.com/adriansyahyasin/status/161...,Retweet


In [72]:
edge.sort_values('Created at')['Created at']

0      2023-01-19 00:00:27
1      2023-01-19 00:00:50
2      2023-01-19 00:00:53
3      2023-01-19 00:00:58
4      2023-01-19 00:01:41
               ...        
5619   2023-01-25 20:36:55
5620   2023-01-25 22:10:14
5621   2023-01-25 22:57:37
5622   2023-01-25 22:59:35
5623   2023-01-25 23:02:22
Name: Created at, Length: 5624, dtype: datetime64[ns]

In [74]:
node_ls_id = edge['Source'].tolist() + edge['Target'].tolist()
node_ls_label = edge['Source Label'].tolist() + edge['Target Label'].tolist()

node = pd.DataFrame(list(zip(node_ls_id,node_ls_label)),columns=['Id','Label'])
node = node.drop_duplicates().reset_index(drop=True)
node

,Id,Label
0,wanderfulgerll,wanderfulgerll
1,dennyidi,dennyidi
2,ilhammufc,ilhammufc
3,icasaurus,icasaurus
4,gajahahahahaha,gajahahahahaha
...,...,...
5124,hrsdn,hrsdn
5125,sedikitau,sedikitau
5126,geloraco,geloraco
5127,MedMezghani,MedMezghani


In [75]:
edge.to_csv('/content/drive/MyDrive/Intro SNA/Assignment/Data/SNA__mobil_listrik_edge.csv',index=False)
node.to_csv('/content/drive/MyDrive/Intro SNA/Assignment/Data/SNA__mobil_listrik_node.csv',index=False)